# Farina in Natura - estratégia

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime



In [2]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [4]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados

In [6]:
# Convert date to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   venda       25992 non-null  float64       
 1   data_hora   25992 non-null  datetime64[ns]
 2   cliente     25992 non-null  object        
 3   mesa        22779 non-null  object        
 4   item        25922 non-null  float64       
 5   produto     25992 non-null  object        
 6   quantidade  25992 non-null  float64       
 7   valor_unit  25992 non-null  object        
 8   desconto    25992 non-null  float64       
 9   total_item  25992 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 2.0+ MB


In [13]:
# Convert the "item" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['item'] = np.int64(transactions['item'])

In [17]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   venda       25992 non-null  float64       
 1   data_hora   25992 non-null  datetime64[ns]
 2   cliente     25992 non-null  object        
 3   mesa        22779 non-null  object        
 4   item        25998 non-null  int64         
 5   produto     25992 non-null  object        
 6   quantidade  25998 non-null  int64         
 7   valor_unit  25992 non-null  object        
 8   desconto    25992 non-null  float64       
 9   total_item  25992 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 2.0+ MB


,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,2022-02-01 08:13:00,CONSUMIDOR,M 1,119,SUCO DE LARANJA 400ML,1,"14,5",0.0,"14,5"
1,460.0,2022-02-01 08:13:00,CONSUMIDOR,M 1,121,PAO SOURDOUGH OVOS,1,"15,5",0.0,"15,5"
2,461.0,2022-02-01 08:14:00,CONSUMIDOR,M 2,119,SUCO DE LARANJA 400ML,1,"14,5",0.0,"14,5"
3,461.0,2022-02-01 08:14:00,CONSUMIDOR,M 2,121,PAO SOURDOUGH OVOS,1,"15,5",0.0,"15,5"
4,465.0,2022-02-01 09:01:00,CONSUMIDOR,M 1,46,QUICHE BACON 90G,2,"19,5",0.0,39


In [18]:
# Convert the "item" variable from float64 type to integer type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = pd.to_numeric(transactions['valor_unit'])

ValueError: Unable to parse string "14,5" at position 0